[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/compare-company-financials-python.ipynb)

# Compare Company Financials with Python -- Free, No API Key

Use **edgartools** to compare financial data across public companies using SEC filings -- completely free, no API key or paid subscription required. Build side-by-side comparisons of revenue, profitability, and financial health from official SEC data.

**What you'll learn:**
- Compare key financial metrics across multiple companies
- Calculate profitability ratios (profit margin, ROA, ROE)
- Compare revenue growth rates
- View multi-year income statements side-by-side
- Export comparison data to pandas DataFrames

## Install edgartools

In [ ]:
!pip install -U edgartools

## Setup

The SEC requires all automated tools to identify themselves. Replace the email below with your own -- any valid email works.

In [ ]:
import pandas as pd
from edgar import *

# The SEC requires you to identify yourself (any email works)
set_identity("your.name@example.com")

## Compare Big Tech Financials in 5 Lines

Pull the same financial metrics from multiple companies and compare them side-by-side:

In [ ]:
tickers = ["NVDA", "MSFT", "AAPL", "GOOG"]

print(f"{'Company':<10} {'Revenue':>20} {'Net Income':>20} {'Total Assets':>20}")
print("-" * 74)
for ticker in tickers:
    fin = Company(ticker).get_financials()
    print(f"{ticker:<10} ${fin.get_revenue():>19,.0f} ${fin.get_net_income():>19,.0f} ${fin.get_total_assets():>19,.0f}")

## Build a Comparison DataFrame

Use `get_financial_metrics()` to pull all available metrics at once and build a DataFrame:

In [ ]:
tickers = ["NVDA", "MSFT", "AAPL", "GOOG"]

rows = []
for ticker in tickers:
    metrics = Company(ticker).get_financials().get_financial_metrics()
    metrics["ticker"] = ticker
    rows.append(metrics)

comparison = pd.DataFrame(rows).set_index("ticker")
comparison[["revenue", "net_income", "total_assets", "stockholders_equity"]]

## Calculate Profitability Ratios

Compute profit margin, return on assets (ROA), and return on equity (ROE) from the extracted metrics:

In [ ]:
tickers = ["NVDA", "MSFT", "AAPL", "GOOG"]
rows = []

for ticker in tickers:
    fin = Company(ticker).get_financials()
    rev = fin.get_revenue()
    ni = fin.get_net_income()
    ta = fin.get_total_assets()
    eq = fin.get_stockholders_equity()

    rows.append({
        "Ticker": ticker,
        "Revenue ($B)": rev / 1e9,
        "Net Income ($B)": ni / 1e9,
        "Profit Margin %": ni / rev * 100,
        "ROA %": ni / ta * 100,
        "ROE %": ni / eq * 100,
    })

ratios = pd.DataFrame(rows).set_index("Ticker").round(1)
ratios

## Compare Revenue Growth

Use `get_facts()` to access multi-year financial statements, then compute year-over-year revenue growth:

In [ ]:
tickers = ["NVDA", "MSFT", "AAPL", "GOOG"]
rows = []

for ticker in tickers:
    facts = Company(ticker).get_facts()
    df = facts.income_statement().to_dataframe()
    rev_row = df[df["label"] == "Total Revenue"]
    fy_cols = [c for c in df.columns if c.startswith("FY")]

    if len(rev_row) > 0 and len(fy_cols) >= 2:
        latest = rev_row[fy_cols[0]].values[0]
        prior = rev_row[fy_cols[1]].values[0]
        growth = (latest - prior) / prior * 100
        rows.append({
            "Ticker": ticker,
            "Period": fy_cols[0],
            "Revenue ($B)": latest / 1e9,
            "Prior Revenue ($B)": prior / 1e9,
            "Growth %": growth,
        })

growth_df = pd.DataFrame(rows).set_index("Ticker").round(1)
growth_df

## View Any Company's Multi-Year Income Statement

The `get_facts()` approach gives you complete multi-year financial statements for any company. Compare two companies' income statements:

In [ ]:
# NVIDIA's multi-year income statement
Company("NVDA").get_facts().income_statement()

In [ ]:
# Microsoft's multi-year income statement
Company("MSFT").get_facts().income_statement()

## Compare Balance Sheet Strength

Build a quick financial health comparison with current ratio and debt-to-assets:

In [ ]:
tickers = ["NVDA", "MSFT", "AAPL", "GOOG"]
rows = []

for ticker in tickers:
    fin = Company(ticker).get_financials()
    ca = fin.get_current_assets()
    cl = fin.get_current_liabilities()
    ta = fin.get_total_assets()
    tl = fin.get_total_liabilities()

    rows.append({
        "Ticker": ticker,
        "Current Assets ($B)": ca / 1e9,
        "Current Liab ($B)": cl / 1e9,
        "Current Ratio": ca / cl,
        "Total Assets ($B)": ta / 1e9,
        "Debt-to-Assets %": tl / ta * 100,
    })

health = pd.DataFrame(rows).set_index("Ticker").round(2)
health

## Why EdgarTools?

EdgarTools is free and open-source. Compare building financial comparisons:

**With edgartools (free, no API key):**
```python
for ticker in ["NVDA", "MSFT", "AAPL"]:
    fin = Company(ticker).get_financials()
    print(ticker, fin.get_revenue(), fin.get_net_income())
```

**Typical paid API approach ($50+/month, API key required):**
```python
from sec_api import XbrlApi
api = XbrlApi(api_key="YOUR_PAID_API_KEY")
# ... make 3 separate API calls, parse JSON, find the right XBRL tags,
# ... handle different tag names across companies, align fiscal years manually
```

With edgartools, financial comparisons use consistent Python methods that work the same across all companies -- no tag hunting, no API key, no monthly fee.

## Quick Reference

```python
from edgar import *
import pandas as pd
set_identity("your.name@example.com")

# ── Latest metrics (from most recent filing) ──
fin = Company("NVDA").get_financials()
fin.get_revenue()              # Revenue
fin.get_net_income()           # Net income
fin.get_total_assets()         # Total assets
fin.get_financial_metrics()    # All metrics as dict

# ── Multi-company comparison ──
rows = []
for ticker in ["NVDA", "MSFT", "AAPL"]:
    metrics = Company(ticker).get_financials().get_financial_metrics()
    metrics["ticker"] = ticker
    rows.append(metrics)
df = pd.DataFrame(rows).set_index("ticker")

# ── Multi-year statements ──
facts = Company("NVDA").get_facts()
facts.income_statement()       # Multi-year income statement
facts.balance_sheet()          # Multi-year balance sheet
```

## What's Next

You've learned how to compare company financials with Python. Here are related tutorials:

- [Extract Revenue and Earnings from SEC Filings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/extract-revenue-earnings-python.ipynb)
- [Extract Financial Statements from SEC Filings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/Viewing-Financial-Statements.ipynb)
- [Parse XBRL Financial Data from SEC EDGAR](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/Reading-Data-From-XBRL.ipynb)
- [Download and Parse 10-K Annual Reports](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/download-10k-annual-report-python.ipynb)

**Resources:**
- [EdgarTools Documentation](https://edgartools.readthedocs.io/)
- [GitHub Repository](https://github.com/dgunning/edgartools)
- [PyPI Package](https://pypi.org/project/edgartools/)

---

## Support EdgarTools

If you found this tutorial helpful, here are a few ways to support the project:

- **Star the repo** -- [github.com/dgunning/edgartools](https://github.com/dgunning/edgartools) -- it helps others discover edgartools
- **Visit edgartools.io** -- [edgartools.io](https://www.edgartools.io/) -- for more tutorials, articles, and updates
- **Report issues** -- found a bug or have a feature idea? [Open an issue](https://github.com/dgunning/edgartools/issues)
- **Share this notebook** -- know someone who works with SEC data? Send them the Colab link

*edgartools is free, open-source, and community-driven. No API key or paid subscription required.*